In [8]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import product
import os

BOARD_SIZE = 100
LADDER_LENGTH = 5
SNAKE_LENGTH = 5
NUM_SIMULATIONS = 1000

In [9]:
import os
import random
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

BOARD_SIZE = 100
LADDER_LENGTH = 5
SNAKE_LENGTH = 5
NUM_SIMULATIONS = 1000

def generate_board(num_snakes, num_ladders):
    """Generates a valid game board with the given number of snakes and ladders."""
    snakes_and_ladders = {}
    iterations = 0
    while True:
        iterations += 1
        potential_ladders = random.sample(
            range(1, BOARD_SIZE - LADDER_LENGTH), num_ladders
        )
        potential_snakes = random.sample(
            range(SNAKE_LENGTH + 1, BOARD_SIZE), num_snakes
        )
        snakes_and_ladders.clear()
        for ladder_start in potential_ladders:
            ladder_end = ladder_start + LADDER_LENGTH
            if ladder_end not in snakes_and_ladders:
                snakes_and_ladders[ladder_start] = ladder_end
        for snake_start in potential_snakes:
            snake_end = snake_start - SNAKE_LENGTH
            if snake_end not in snakes_and_ladders:
                snakes_and_ladders[snake_start] = snake_end
        overlaps = [
            pos
            for pos in snakes_and_ladders
            if list(snakes_and_ladders.values()).count(pos) > 1
        ]
        if not overlaps:
            break
        for overlap in overlaps:
            if random.random() < 0.5:
                snake_start = [
                    key
                    for key, value in snakes_and_ladders.items()
                    if value == overlap and key > overlap
                ][0]
                del snakes_and_ladders[snake_start]
            else:
                ladder_start = [
                    key
                    for key, value in snakes_and_ladders.items()
                    if value == overlap and key < overlap
                ][0]
                del snakes_and_ladders[ladder_start]
    return snakes_and_ladders, iterations

In [10]:
def play_game(snakes_and_ladders):
    """Simulates a single game of Snakes and Ladders."""
    position = 0
    moves = 0
    while position < BOARD_SIZE:
        dice_roll = random.SystemRandom().randint(1, 6)
        position += dice_roll
        moves += 1
        if position in snakes_and_ladders:
            position = snakes_and_ladders[position]
        if position > BOARD_SIZE:
            position = BOARD_SIZE
    return moves

In [11]:
def analyze_board_configuration(num_snakes, num_ladders, num_realizations=10):
    """
    Analyzes a specific configuration of snakes and ladders over multiple realizations.
    """
    total_iterations = 0
    all_game_times = []

    # Create directory for this snake count if it doesn't exist
    if not os.path.exists(f"snakes_{num_snakes}"):
        os.makedirs(f"snakes_{num_snakes}")

    fig, axes = plt.subplots(2, 5, figsize=(20, 10))
    fig.suptitle(
        f"Snakes: {num_snakes}, Ladders: {num_ladders} - Game Time Distributions"
    )

    # Multiple realizations of the same configuration
    for realization in range(num_realizations):
        board, iterations = generate_board(num_snakes, num_ladders)
        total_iterations += iterations
        game_times = [play_game(board) for _ in range(NUM_SIMULATIONS)]
        all_game_times.append(game_times)

        # Plot frequency distribution for each realization as subplots
        ax = axes.flatten()[realization]
        ax.hist(game_times, bins=20, edgecolor="black")
        ax.set_title(f"Board: {realization+1}")
        ax.set_xlabel("Game Time (Moves)")
        ax.set_ylabel("Frequency")

    # Save the figure with subplots
    plt.tight_layout()
    plt.savefig(f"snakes_{num_snakes}/snakes_{num_snakes}_ladders_{num_ladders}_game_time_distributions.png")
    plt.close()

    # Save raw data
    df_raw = pd.DataFrame(all_game_times).transpose()
    df_raw.columns = [f"Board {i+1}" for i in range(num_realizations)]
    df_raw.to_csv(f"snakes_{num_snakes}/ladders_{num_ladders}.csv", index=False)

    avg_iterations = total_iterations / num_realizations
    avg_game_time = np.mean(all_game_times)
    return avg_iterations, avg_game_time

In [12]:
def run_analysis():
    """Runs the analysis for various combinations of snakes and ladders."""
    snake_range = range(5, 16)
    ladder_range = range(5, 16)
    results = []
    for num_snakes, num_ladders in product(snake_range, ladder_range):
        avg_iterations, avg_game_time = analyze_board_configuration(
            num_snakes, num_ladders
        )
        results.append(
            {
                "num_snakes": num_snakes,
                "num_ladders": num_ladders,
                "avg_iterations": avg_iterations,
                "avg_game_time": avg_game_time,
                "snake_ladder_ratio": num_snakes / num_ladders,
                "snake_ladder_diff": num_snakes - num_ladders,
            }
        )
    return pd.DataFrame(results)

In [13]:
def plot_aggregate_results(results_df):
    """Plots aggregate heatmaps and relative scoring charts."""
    # 1. Average iterations heatmap
    plt.figure(figsize=(8, 6))
    pivot_iterations = results_df.pivot(
        index="num_snakes", columns="num_ladders", values="avg_iterations"
    )
    sns.heatmap(pivot_iterations, annot=True, fmt=".1f")
    plt.title("Average Iterations Required to Generate Valid Board")
    plt.xlabel("Number of Ladders")
    plt.ylabel("Number of Snakes")
    plt.savefig("avg_iterations_heatmap.png")
    plt.close()

    # 2. Average game time heatmap
    plt.figure(figsize=(8, 6))
    pivot_times = results_df.pivot(
        index="num_snakes", columns="num_ladders", values="avg_game_time"
    )
    sns.heatmap(pivot_times, annot=True, fmt=".1f")
    plt.title("Average Game Time (Moves)")
    plt.xlabel("Number of Ladders")
    plt.ylabel("Number of Snakes")
    plt.savefig("avg_game_time_heatmap.png")
    plt.close()

    # 3. Game time vs Snake/Ladder ratio
    plt.figure(figsize=(8, 6))
    plt.scatter(results_df["snake_ladder_ratio"], results_df["avg_game_time"])
    plt.xlabel("Snake/Ladder Ratio")
    plt.ylabel("Average Game Time (Moves)")
    plt.title("Game Time vs Snake/Ladder Ratio")
    plt.savefig("game_time_vs_ratio.png")
    plt.close()

    # 4. Game time vs Snake-Ladder difference
    plt.figure(figsize=(8, 6))
    plt.scatter(results_df["snake_ladder_diff"], results_df["avg_game_time"])
    plt.xlabel("Snakes - Ladders")
    plt.ylabel("Average Game Time (Moves)")
    plt.title("Game Time vs Snake-Ladder Difference")
    plt.savefig("game_time_vs_diff.png")
    plt.close()

In [ ]:
# Run the analysis
if __name__ == "__main__":
    print("Running analysis...")
    results_df = run_analysis()

    # Save the final dataframe
    results_df.to_csv("final_results.csv", index=False)

    print("\nAnalysis Results:")
    print(results_df)

    print("\nGenerating plots...")
    plot_aggregate_results(results_df)